# User Manual for QuEP
## Betatron Test

### Prerequisites: 
Download vs code, put QuEP on vs code, have installed all imports mentioned in plot2DTracks.py and main.py, read over the README file

How to for prerequisites: look up vs code and download (be sure to connect python and jupyter and anything else, download the QuEP code from GitHub and open as a folder in vs code, make an 'imports' python file and install all modules using conda install

### Starting actual betatron test:

#### Input
The first thing needed to run anything in QuEP is an input file. This will specify things like number of electrons, shape of the beam, momentum, and initial position

To make an input file I recommend creating a copy of a pre-existing file and altering it. Any file in the 'input' folder will work. To make a copy right click on a file, press copy, then use ctrl + v to paste. You can then rename this file to whaterver you want it to be

To edit your input file click on it so that the file shows up in the vs code viewer. Most of what is present in the input file in discussed in the README file so please reference that while editing your file. I will show my example file with what each parameter should be for the betatron test here and a brief explanation why

In [1]:
simulation_name = 'QUASI3D'  
#Explanation in README file. All simulations should have this as simulation name

shape = 'single'    
#betatron test only uses a singlular electron. See other options in README file for other sims

iterations = 500000
#Iterations refers to the amount of times the code is run. The input conditions are put into equations using main.py.
#A high number of iterations will have less space between each 't' value used which will have increased clarity in graphs

mode = 0 
#mode 0 is the wakefield, mode 1 is the laser, mode -1 is both. Betatron is to see electron in wakefield

fname = "ashley_input.npz"
#main.py will put the data from the simulation into this file. name it whatever you want your data file to be

debugmode = True
#debugmode should only be true for single electron simulations (I'm not actually sure why)


# Probe centered at the following initial coordinates (in c/w_p):
#See figure 3.1 in Evan Trommer's thesis. The goal is to put the electron in line with the peak of the second bump of the field
#so while xi_c here is fairly fixed y_c can be any range of values still within the bubble
x_c = 0 # Start within region of field # 2.4 = maximum x_c
y_c = 0.25
xi_c = -8.3

# Initial momentum
#(I don't really remember I think it's just because it's like solely transverse probing)
px_0 = 0 # Make sure it goes towards the screen!
py_0 = 0
pz_0 = 110

# Screen Distances (from z-axis of plasma cell, in mm):
x_s = [10, 50, 100, 250, 500]

# Shape Parameters (Radius or Side Length, in c/w_p):
s1 = 1 # In y
s2 = 1 # In xi

# Densities
#you only have a single electron so there should just be one when you count on any axis
ydensity = 1
xidensity = 1
xdensity = 1 # Probe width - single layer
resolution = None 


#### Moving the data
Once the input file is done you run main.py using it so that you can get the data file to process into the graphs. 
To run main.py you use the terminal at the bottom of the vs code screen and type: 'python main.py input.urInputFile' and enter
This will create two files inside of the QuEP folder located on the left side of the screen. One will be your npz file and one will be your debug file. In order to use these you must move them to the data folder. The command to put in your terminal is 'mv urFileName.npz(or obj for debug) data/'.


#### Post Processing (making the graphs!)
To run the post processing step you just enter 'python index-mp.py input.urInputFile' in your terminal. (But come back here after looking over the next few boxes)

##### Progressbar errors
Before running the final part of the betatron test you should be prepared to make a few edits to some files index-mp.py uses. Previously QuEP used a lot of a module called progressbar that for some reason doesn't work anymore. The easiest (though most time consuming way) to find all the progressbar parts is to just run the code and look at each error message. 
For each progressbar error go into the referenced file and comment out 'import progressbar' and check if progressbar is used anywhere in the code. If it is it likely is of the form 'for i in progressbar.progressbar(range(0,len(x_0)), redirect_stout=False):'. To change this all you keep is the 'for i in range(0,len(x_0)):' and don't forget to save the code after each edit.

##### Plot2DTracks edits
I've also included my plot2DTracks.py file below as there will probably be a few edits to make to that too. The most notable edit is in line ~41 I changed the GitHub version's 

"ax1.plot(x_dat[i,:], z_dat[i,:], 'k', label='$Z-X Trajectory') # Want vertical axis as y" 

to   "ax1.plot(z_dat[i,:], y_dat[i,:], 'k', label='$Z-X Trajectory') # Want vertical axis as y"

In [ ]:
# Script for generating 2D plots of electron trajectories with option for plotting force

import numpy as np
import matplotlib.colors as col
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.ticker as ticker
from mpl_toolkits.mplot3d import Axes3D
import pdb

from numpy.core.fromnumeric import size
plt.rcParams.update({'font.size': 16})

plotYForce = True # Plot transverse force with trajectories, not useful for many trajectories
plotZForce = True # Plot force along WF propagation

#large_size = 12

#plt.rc('ytick', labelsize=large_size)
#plt.rc('axes', labelsize=large_size)

def make_patch_spines_invisible(ax):
    ax.set_frame_on(True)
    ax.patch.set_visible(False)
    for sp in ax.spines.values():
        sp.set_visible(False)

def plot(x_dat,y_dat,z_dat,xi_dat,Fx_dat,Fy_dat,Fz_dat,px_dat,py_dat,sim_name,shape_name,s1,s2,noElec,fname):

# 2D: Z-X, constrained to blowout regime THIS IS FIRST PLOT
    fig1 = plt.figure(1)
    ax1 = plt.axes()
    ax1.set_xlabel("Z ($c/\omega_p$)")
    ax1.set_ylabel("X ($c/\omega_p$)")
    ax1.set_xlim(0,600)
    ax1.tick_params(axis='y', labelcolor='k')
    ax1.set_title("Electron Trajectories through Blowout Regime")

    for i in range(0, noElec):
        ax1.plot(z_dat[i,:], y_dat[i,:], 'k', label='$Z-X Trajectory') # Want vertical axis as y

    if (plotZForce):
        ax1_f = ax1.twinx()
        ax1_f.set_ylabel("$F_z$ ($m_e c \omega_p$)")
        ax1_f.yaxis.label.set_color('C0')
        ax1_f.tick_params(axis='y', labelcolor='C0', colors='C0')

        for i in range(0, noElec):
            ax1_f.plot(z_dat[i,:], Fz_dat[i,:], 'C0', label='Z Force')

        fig1.legend(bbox_to_anchor=(0.88, 0.94), bbox_transform=plt.gcf().transFigure)

# 2D: Y-X THIS IS FOR SECOND PLOT
    fig2, ax2 = plt.subplots(1,figsize=(15,10),dpi=300)
    fig2.subplots_adjust(right=0.75)

    for i in range(0, noElec):
        #y_dat[i,:] = [y/0.65 for y in y_dat[i,:]]
        ax2.plot(x_dat[i,:], y_dat[i,:], 'k', label='Y-X Electron Trajectory') # Want vertical axis as y
        ax2.set_xlim(-1.5,1.5) #OG was (-10,10)
        ax2.set_ylim(-0.2,0.7)
    ax2.set_xlabel("X ($c/\omega_p$)")
    ax2.set_ylabel("Y/$R_b$ ($c/\omega_p$)")
    ax2.set_title("Electron Trajectory through Blowout Regime")

    if (plotYForce):
        Fy_ax = ax2.twinx()
        px_ax = ax2.twinx()
        py_ax = ax2.twinx()

        px_ax.spines["right"].set_position(("axes",1.15))
        make_patch_spines_invisible(px_ax)
        px_ax.spines["right"].set_visible(True)
        py_ax.spines["right"].set_position(("axes",1.3))
        make_patch_spines_invisible(py_ax)
        py_ax.spines["right"].set_visible(True)

        for i in range(0, noElec):
            Fy_ax.plot(x_dat[i,:], Fy_dat[i,:], 'C0', label='Transverse Electric Force, $F_y$')
            px_ax.plot(x_dat[i,:], px_dat[i,:], 'C1', label='Momentum in X')
            py_ax.plot(x_dat[i,:], py_dat[i,:], 'C2', label='Momentum in Y')
        Fy_ax.set_ylabel("$F_y$ ($m_e c \omega_p$)")
        px_ax.set_ylabel("$p_x (m_e c)$")
        py_ax.set_ylabel("$p_y (m_e c)$")

        Fy_ax.yaxis.label.set_color('C0')
        px_ax.yaxis.label.set_color('C1')
        py_ax.yaxis.label.set_color('C2')

        tkw = dict(size=4, width=1.5)
        ax2.tick_params(axis='y', colors='k', **tkw)
        Fy_ax.tick_params(axis='y', colors='C0', **tkw)
        px_ax.tick_params(axis='y', colors='C1', **tkw)
        py_ax.tick_params(axis='y', colors='C2', **tkw)
        ax2.tick_params(axis='x', **tkw)

        ax2.grid()
        fig2.legend(bbox_to_anchor=(0.3, 0.8), bbox_transform=plt.gcf().transFigure)

    fig1.tight_layout()
    #fig1.show()
    fig2.tight_layout()
    fig2.savefig(f"eProbe-Trajectories_{fname}.png",transparent=False)
    fig1.savefig(f"eProbe-Trajectories1_{fname}.png",transparent=False)

### The Graphs!
Once you've made the progressbar and plot2DTracks edits you are ready to run the 'making the graphs' step.
After running this you should see two files pop up in the left hand side directory of your folder on vs code. These two purple files are the images of your graphs. Once you have these copied and saved wherever you want you're free to delete them from your directory and you are finished!